# Load the Pretrained Model and the Dataset

In [1]:
import sys

sys.path.append("../..")
sys.path.append("../../../")

In [2]:
import paddle
import paddlenlp
from paddlenlp.transformers import ErnieForSequenceClassification, ErnieTokenizer

MODEL_NAME = 'ernie-1.0'

model = ErnieForSequenceClassification.from_pretrained(MODEL_NAME, num_classes=2)
tokenizer = ErnieTokenizer.from_pretrained(MODEL_NAME)

[2022-05-17 16:32:27,097] [    INFO] - Already cached /home/zhangshuai/.paddlenlp/models/ernie-1.0/ernie_v1_chn_base.pdparams
W0517 16:32:27.100387 29316 device_context.cc:447] Please NOTE: device: 0, GPU Compute Capability: 7.0, Driver API Version: 11.4, Runtime API Version: 10.2
W0517 16:32:27.105998 29316 device_context.cc:465] device: 0, cuDNN Version: 8.2.
[2022-05-17 16:32:32,056] [    INFO] - Already cached /home/zhangshuai/.paddlenlp/models/ernie-1.0/vocab.txt


In [3]:
from paddlenlp.datasets import load_dataset

DATASET_NAME = 'chnsenticorp'
train_ds, dev_ds, test_ds = load_dataset(DATASET_NAME, splits=["train", "dev", "test"])

# Prepare the Model
## Train the model

In [ ]:
from assets.utils import training_model

training_model(model, tokenizer, train_ds, dev_ds, save_dir=f'../../assets/{DATASET_NAME}-{MODEL_NAME}')


## Or Load the trained model

In [4]:
# Load the trained model.
!wget --no-check-certificate -c https://trustai.bj.bcebos.com/chnsenticorp-ernie-1.0.tar
!tar -xvf ./chnsenticorp-ernie-1.0.tar -C ../../assets/
!rm ./chnsenticorp-ernie-1.0.tar

state_dict = paddle.load(f'../../assets/{DATASET_NAME}-{MODEL_NAME}/model_state.pdparams')
model.set_dict(state_dict)

--2022-05-17 16:32:32--  https://trustai.bj.bcebos.com/chnsenticorp-ernie-1.0.tar
Resolving trustai.bj.bcebos.com (trustai.bj.bcebos.com)... 10.70.0.165
Connecting to trustai.bj.bcebos.com (trustai.bj.bcebos.com)|10.70.0.165|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 399595520 (381M) [application/x-tar]
Saving to: ‘chnsenticorp-ernie-1.0.tar’

100%[======================================>] 399,595,520  121MB/s   in 3.2s   

2022-05-17 16:32:35 (121 MB/s) - ‘chnsenticorp-ernie-1.0.tar’ saved [399595520/399595520]

chnsenticorp-ernie-1.0/
chnsenticorp-ernie-1.0/tokenizer_config.json
chnsenticorp-ernie-1.0/vocab.txt
chnsenticorp-ernie-1.0/model_state.pdparams
chnsenticorp-ernie-1.0/model_config.json


# See the prediciton results

In [5]:
from assets.utils import predict

test_data = [{'text': '本来不想评价了，但为了携程的携粉们，还是说一下，这称不上是九点，细说就真没必要了，就一个字：差'}]

label_map = {0: 'negative', 1: 'positive'}

batch_size = 32

results = predict(
    model, test_data, tokenizer, label_map, batch_size=batch_size)

for idx, text in enumerate(test_data):
    print('Data: {} \t Lable: {}'.format(text, results[idx]))

Data: {'text': '本来不想评价了，但为了携程的携粉们，还是说一下，这称不上是九点，细说就真没必要了，就一个字：差'} 	 Lable: negative


# Prepare for Interpretations

In [6]:
from functools import partial

from paddlenlp.data import Stack, Tuple, Pad

from assets.utils import create_dataloader, convert_example


batch_size = 32
max_seq_length = 128

trans_func = partial(
        convert_example,
        tokenizer=tokenizer,
        max_seq_length=max_seq_length,
        is_test=True,
    )
batchify_fn = lambda samples, fn=Tuple(
    Pad(axis=0, pad_val=tokenizer.pad_token_id),  # input
    Pad(axis=0, pad_val=tokenizer.pad_token_type_id),  # segment
): [data for data in fn(samples)]
train_data_loader = create_dataloader(
    train_ds,
    mode='train',
    batch_size=batch_size,
    batchify_fn=batchify_fn,
    trans_fn=trans_func,
    shuffle=False)

# Feature similarity Interpreter

In [7]:
from trustai.interpretation.example_level.method.feature_similarity import FeatureSimilarityModel

# classifier_layer_name is the layer name of the last output layer
feature_sim = FeatureSimilarityModel(model, train_data_loader, classifier_layer_name='classifier')

Extracting feature for given dataloader, it will take some time...


In [8]:
from assets.utils import create_dataloader_from_scratch

test_data = [{'text': '本来不想评价了，但为了携程的携粉们，还是说一下，这称不上是九点，细说就真没必要了，就一个字：差'}]

# process text to model input
test_dataloader = create_dataloader_from_scratch(test_data, tokenizer)

sim_fn = "cos"
sample_num = 3
predict_labels, most_sim_examples = feature_sim.interpret(
        test_dataloader, sample_num=sample_num, sim_fn=sim_fn
    )

for i in range(len(test_data)):
    print("test data")
    print(f"text: {test_data[i]['text']}\tpredict label: {predict_labels[i]}")
    print("most similar examples")
    for example in most_sim_examples[i][0]:
        print(
            f"text: {train_ds.data[example]['text']}\tgold label: {train_ds.data[example]['label']}"
        )
    print("most dissimilar examples")
    for example in most_sim_examples[i][1]:
        print(
            f"text: {train_ds.data[example]['text']}\tgold label: {train_ds.data[example]['label']}"
        )
    print()

Extracting feature for given dataloader, it will take some time...
test data
text: 本来不想评价了，但为了携程的携粉们，还是说一下，这称不上是九点，细说就真没必要了，就一个字：差	predict label: 0
most similar examples
text: 我选分期付款，在上海扣的款，在上海开的发票，东西却从北京用快递发出，真是舍近求远，害的我等了一星期才收到货，真是脑残！	gold label: 0
text: 看到评价那么高，就买了，但女儿不喜欢，我看了一下，也不喜欢，不知所云，汽车什么的，都是过时的，或生活中没有的。不明白，为什么有这么高的评价。	gold label: 0
text: 看到评价那么高，就买了，但女儿不喜欢，我看了一下，也不喜欢，不知所云，汽车什么的，都是过时的，或生活中没有的。不明白，为什么有这么高的评价。	gold label: 0
most dissimilar examples
text: 昨晚看着看着就睡着了，今天早晨醒来就立马抓起继续啃，正逢小说结尾部分，也正如作者的期望，我被吓了一跳。这是我读东野圭吾小说最意料不到的事实。爱有很多种，可是当它变成某种负担时，还算爱吗？爱有多种表现形式，当某种行为让对方感觉负罪时，这还能叫做爱吗？爱可以很长久，当这种永恒变成一种恶梦时，还能归于最初的爱吗？爱可以深入骨髓，无法割除，当它接受无法预知的变数时，还能始终如一吗？	gold label: 1
text: 她们是一群神秘、美丽的女人，从上古的嫘祖，到末代的婉如，每一个都吸引了无数人。我们翻开历史，她们似乎是男人的陪衬，可是当你细细研读，你会发现她们是不可或缺的人物，是每一个朝代最美丽的花朵，也是每一位皇帝的明珠。她们有的与丈夫情投意合，助男人打下一片天下，有的共享荣华，帮皇帝创造盛世，有的颠沛流离，与失意的他同甘共苦，有的……，这一切不仅仅是故事，更是历史，神秘、华美的历史。	gold label: 1
text: 受重建轻管思想的影响，造成水利工程管理研究十分薄弱，涉及工程管理，特别是河道管理类的书籍就更少了。郑教授长期从事河道管理工作，有十分丰富的管理经验和理论修养。该书系统阐述了河道的基本理论，国内外先进管理模式和发展方向，对我国河道管理、法规

In [9]:
from assets.utils import create_dataloader_from_scratch

test_data = [{'text': '本来不想评价了，但为了携程的携粉们，还是说一下，这称不上是九点，细说就真没必要了，就一个字：差'}]

# process text to model input
test_dataloader = create_dataloader_from_scratch(test_data, tokenizer)

sim_fn = "dot"
sample_num = 3
predict_labels, most_sim_examples = feature_sim.interpret(
        test_dataloader, sample_num=sample_num, sim_fn=sim_fn
    )

for i in range(len(test_data)):
    print("test data")
    print(f"text: {test_data[i]['text']}\tpredict label: {predict_labels[i]}")
    print("most similar examples")
    for example in most_sim_examples[i][0]:
        print(
            f"text: {train_ds.data[example]['text']}\tgold label: {train_ds.data[example]['label']}"
        )
    print()
    print("most dissimilar examples")
    for example in most_sim_examples[i][1]:
        print(
            f"text: {train_ds.data[example]['text']}\tgold label: {train_ds.data[example]['label']}"
        )
    print()

Extracting feature for given dataloader, it will take some time...
test data
text: 本来不想评价了，但为了携程的携粉们，还是说一下，这称不上是九点，细说就真没必要了，就一个字：差	predict label: 0
most similar examples
text: 虽至于另外的那些更是不值一提，选来选去三星的这款不错。然以前用的三星不好，但这次不错	gold label: 0
text: 京东太会糊弄人了，我选定的圆通 ，竟然给发了免费的经济EMS害我多花了8块钱！	gold label: 0
text: 什么全球最大的中文网上商城，我呸！乱发货不说，客服态度差、处理速度慢、不主动。想忽忧消费者呀？	gold label: 0

most dissimilar examples
text: 现代许多年轻人喜欢读那些女人气的散文，对民俗作品不屑一顾，其实，民俗民情才是我们中国文化的根。花再娇艳也要有根的支撑啊。太湖实在是茶文化的故乡，就算日本的茶道也起源于中国。更何况茶圣陆羽的茶经也在太湖写成。太湖畔的惠泉山水，曾经让唐朝宰相李德裕，像杨贵妃千里传荔枝一样的让快马从无锡送去惠泉山水到长安，只为煮茶品茗。太湖茶俗，实在影响了中国数千年啊。这本书，你值得拥有。	gold label: 1
text: 她们是一群神秘、美丽的女人，从上古的嫘祖，到末代的婉如，每一个都吸引了无数人。我们翻开历史，她们似乎是男人的陪衬，可是当你细细研读，你会发现她们是不可或缺的人物，是每一个朝代最美丽的花朵，也是每一位皇帝的明珠。她们有的与丈夫情投意合，助男人打下一片天下，有的共享荣华，帮皇帝创造盛世，有的颠沛流离，与失意的他同甘共苦，有的……，这一切不仅仅是故事，更是历史，神秘、华美的历史。	gold label: 1
text: 受重建轻管思想的影响，造成水利工程管理研究十分薄弱，涉及工程管理，特别是河道管理类的书籍就更少了。郑教授长期从事河道管理工作，有十分丰富的管理经验和理论修养。该书系统阐述了河道的基本理论，国内外先进管理模式和发展方向，对我国河道管理、法规现状进行了梳理，提出问题和措施问题，并对涉河项目行政许可、河道具体管理提出具体方法和指导，理念新，实用性强，非常有价值！江苏水利 刘	gold l

In [10]:
from assets.utils import create_dataloader_from_scratch

test_data = [{'text': '本来不想评价了，但为了携程的携粉们，还是说一下，这称不上是九点，细说就真没必要了，就一个字：差'}]

# process text to model input
test_dataloader = create_dataloader_from_scratch(test_data, tokenizer)

sim_fn = "euc"
sample_num = 3
predict_labels, most_sim_examples = feature_sim.interpret(
        test_dataloader, sample_num=sample_num, sim_fn=sim_fn
    )
print(most_sim_examples)
for i in range(len(test_data)):
    print("test data")
    print(f"text: {test_data[i]['text']}\tpredict label: {predict_labels[i]}")
    print("most similar examples")
    for example in most_sim_examples[i][0]:
        print(
            f"text: {train_ds.data[example]['text']}\tgold label: {train_ds.data[example]['label']}"
        )
    print("most dissimilar examples")
    for  example in most_sim_examples[i][1]:
        print(
            f"text: {train_ds.data[example]['text']}\tgold label: {train_ds.data[example]['label']}"
        )
    print()

Extracting feature for given dataloader, it will take some time...
[([3256, 9517, 3629], [5842, 8737, 3627])]
test data
text: 本来不想评价了，但为了携程的携粉们，还是说一下，这称不上是九点，细说就真没必要了，就一个字：差	predict label: 0
most similar examples
text: 我选分期付款，在上海扣的款，在上海开的发票，东西却从北京用快递发出，真是舍近求远，害的我等了一星期才收到货，真是脑残！	gold label: 0
text: 看到评价那么高，就买了，但女儿不喜欢，我看了一下，也不喜欢，不知所云，汽车什么的，都是过时的，或生活中没有的。不明白，为什么有这么高的评价。	gold label: 0
text: 看到评价那么高，就买了，但女儿不喜欢，我看了一下，也不喜欢，不知所云，汽车什么的，都是过时的，或生活中没有的。不明白，为什么有这么高的评价。	gold label: 0
most dissimilar examples
text: 现代许多年轻人喜欢读那些女人气的散文，对民俗作品不屑一顾，其实，民俗民情才是我们中国文化的根。花再娇艳也要有根的支撑啊。太湖实在是茶文化的故乡，就算日本的茶道也起源于中国。更何况茶圣陆羽的茶经也在太湖写成。太湖畔的惠泉山水，曾经让唐朝宰相李德裕，像杨贵妃千里传荔枝一样的让快马从无锡送去惠泉山水到长安，只为煮茶品茗。太湖茶俗，实在影响了中国数千年啊。这本书，你值得拥有。	gold label: 1
text: 她们是一群神秘、美丽的女人，从上古的嫘祖，到末代的婉如，每一个都吸引了无数人。我们翻开历史，她们似乎是男人的陪衬，可是当你细细研读，你会发现她们是不可或缺的人物，是每一个朝代最美丽的花朵，也是每一位皇帝的明珠。她们有的与丈夫情投意合，助男人打下一片天下，有的共享荣华，帮皇帝创造盛世，有的颠沛流离，与失意的他同甘共苦，有的……，这一切不仅仅是故事，更是历史，神秘、华美的历史。	gold label: 1
text: 受重建轻管思想的影响，造成水利工程管理研究十分薄弱，涉及工程管理，特别是河道管理类的书籍就更少了。郑教授长期从事河道管理工作，有十分丰富的管理经验和理论修养。